# MIT License

Copyright 2020 Pablo Duboue

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

This code is derived from the Case Study in Chapter 10, see https://github.com/DrDub/artfeateng

<a href="https://colab.research.google.com/github/DrDub/riiaa20_ws25_feateng_space_time/blob/master/notebooks/1_Spatial_Landmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import os
import subprocess
import sys

# colab bit, adapted from https://github.com/riiaa/riiaa19_workshop_template

GIT_NAME ='riiaa20_ws25_feateng_space_time'
GIT_URL  ='https://github.com/DrDub/{}.git'.format(GIT_NAME)
IN_COLAB = 'google.colab' in sys.modules

def run_cmd(cmd):
    print('Output of "{}":'.format(cmd))
    print(subprocess.run(cmd,stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8'))
        
if IN_COLAB:
    SRC_DIR='.'
    run_cmd('rm -rf sample_data')
    run_cmd('rm -rf {}'.format(GIT_NAME))
    run_cmd('git clone --verbose --progress {}'.format(GIT_URL))
    run_cmd('mv {}/* . '.format(GIT_NAME))
    run_cmd('rm -rf {}'.format(GIT_NAME))
else:
    SRC_DIR='..'
    
print('Using colab? {}, using root directory "{}"'.format(IN_COLAB,SRC_DIR))

Using colab? False, using root directory ".."


# Geographical Features

In GIS features, we would look at using representions based on distance to key points. 

## Example Data

We will be using the migration data through satellite telemetry for African cuckoos in Nigeria, kindly released by [Iwajomo SB et al, 2018 as part of the movebank.org data repository](https://www.datarepository.movebank.org/handle/10255/move.714). It contains 12,563 tracking points for six individuals, from May 29th, 2013 until Jun 28th, 2017.

![Data trajectory](../media/movebank.png)

We will look into the problem of predicting whether a bird will move in the next two hours.

For other potentially useful features inside the data, see its [readme](../data/MoveBank.readme.txt).

In [2]:
# from CELL 21
import os

rows = list()
first = 0
with open(os.path.join(SRC_DIR, "data", "African cuckoo in Nigeria (data from Iwajomo et al. 2018).csv")) as csv:
    header = None
    for row in csv:
        if first < 5:
            print(row)
            first += 1
        cols = row.split(",")
        if header is None:
            header = { w: i for i, w in enumerate(cols) }
        else:
            long = cols[header['location-long']]
            lat  = cols[header['location-lat']]
            if len(long) > 3 and len(lat) > 3:
                rows.append( [ 
                    cols[header['timestamp']],
                    lat,
                    long,
                    cols[header['individual-local-identifier']]
                ])
print("Read", len(rows), "rows")

event-id,visible,timestamp,location-long,location-lat,algorithm-marked-outlier,argos:altitude,argos:best-level,argos:calcul-freq,argos:error-radius,argos:gdop,argos:iq,argos:lat1,argos:lat2,argos:lc,argos:lon1,argos:lon2,argos:nb-mes,argos:nb-mes-120,argos:nopc,argos:orientation,argos:pass-duration,argos:semi-major,argos:semi-minor,argos:sensor-1,argos:sensor-2,argos:sensor-3,argos:sensor-4,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name

4047671215,false,2013-06-02 09:53:01.000,8.94982,9.83805,true,1345.0,-132.0,4.0167940869E8,4500.0,7399,"0",9.83805,9.83805,"B",8.94982,8.94982,2,0,3,159.0,295.0,130425.0,155.0,183,200,3,8,,"argos-doppler-shift","Cuculus gularis","126694","126694","African cuckoo in Nigeria (data from Iwajomo et al. 2018)"

4047671220,false,2013-06-02 13:23:14.000,8.97863,9.87196,true,1396.0,-127.0,4.0167940247E8,1655.0,234,"48",9.87196,9.87196,"0",8.97863,8.97863,6,0,3,80.0,536.0,4171.0,65

We will now split the rows by individual, leaving two individuals for training, two for devtest and one for final testing.

To obtain the classes (moved or not), we parse the time stamps and assign a class depending on the number of seconds between two entries. We will also explode the date and time as individual features and compute a "day in the year" feature.

In [3]:
# from CELL 22
import random

individuals = set(map(lambda x:x[-1], rows))

print("Total individuals:", len(individuals))

random_indiv = list(sorted(individuals))
for indiv in random_indiv:
    print("{}: {:5,}".format(indiv, len(list(filter(lambda x:x[-1] == indiv, rows)))))

random.shuffle(random_indiv, random=random.Random(68).random)

train_indiv      = set(random_indiv[2:4])
test_indiv       = set(random_indiv[0:2])

train_rows = len(list(filter(lambda x:x[-1] in train_indiv, rows)))
print("Total train rows: {:,}".format(train_rows))

test_rows = len(list(filter(lambda x:x[-1] in test_indiv, rows)))
print("Total test rows:  {:,}".format(test_rows))

Total individuals: 5
"126694": 4,568
"126695":   136
"126696": 3,617
"126697": 3,063
"126698":   712
Total train rows: 4,329
Total test rows:  4,704


## EDA

We will use [Folium](https://github.com/python-visualization/folium) to visualize the points on a map.



In [4]:
import folium
import numpy as np

train_coords = np.zeros((train_rows,2),dtype=float)
train_coords = np.array(list(map(lambda x:[ float(x[1]), float(x[2]) ], 
                                 filter(lambda x:x[-1] in train_indiv, rows))))

# center for points
lat_center, lon_center = train_coords[:,0].mean(), train_coords[:,1].mean()

sample_coords = random.Random(42).sample(list(train_coords), 1000)

# Build map 
bird_map = folium.Map(location=(lat_center, lon_center), zoom_start=6, tiles='cartodbpositron', width=640, height=480)

# Plot coordinates using comprehension list
# credit: https://stackoverflow.com/questions/45388412/what-is-the-fastest-way-to-plot-coordinates-on-map-inline-jupyter
[folium.CircleMarker(sample_coords[i], radius=1, 
                     color='#0080bb', fill_color='#0080bb').add_to(bird_map) 
    for i in range(len(sample_coords))]

# Display map in Jupyter
bird_map


## First featurization

In [5]:
# from CELL 23
import re
import datetime
import numpy as np

xtrain = np.zeros((train_rows, 
                   3 + # year, month, day 
                   1 + # day of year
                   3 + # hh, mm, ss
                   2 # lat long
                  ), dtype='float32')
ytrain = np.zeros((train_rows,), dtype='float32')

prev = None
for idx, row in enumerate(filter(lambda x:x[-1] in train_indiv, rows)):
    year, month, day, hh, mm, ss = re.split(" |-|:+", row[0])
    year  = int(year)
    month = int(month)
    day   = int(day)
    hh = int(hh)
    mm = int(hh)
    ss = float(ss)
    dt = datetime.datetime(year, month, day, hh, mm, int(ss))
    try:
        xtrain[idx,:] = [ 
            year, month, day, dt.timetuple().tm_yday, hh, mm, ss,
            float(row[1]), float(row[2])
        ]
    except:
        print(row)
    ts = dt.timestamp()
    if prev is None:
        ytrain[idx] = 0.0
    else:
        if ts - prev > 2 * 60 * 60:
            ytrain[idx] = 1.0
        else:
            ytrain[idx] = 0.0
    prev = ts 
print(np.sum(ytrain))

xtest = np.zeros((test_rows, xtrain.shape[1]), dtype='float32')
ytest = np.zeros((test_rows,), dtype='float32')

prev = None
for idx, row in enumerate(filter(lambda x:x[-1] in test_indiv, rows)):
    year, month, day, hh, mm, ss = re.split(" |-|:+", row[0])
    year  = int(year)
    month = int(month)
    day   = int(day)
    hh = int(hh)
    mm = int(hh)
    ss = float(ss)
    dt = datetime.datetime(year, month, day, hh, mm, int(ss))
    try:
        xtest[idx,:] = [ 
            year, month, day, dt.timetuple().tm_yday, hh, mm, ss,
            float(row[1]), float(row[2])
        ]
    except:
        print(row)
    ts = dt.timestamp()
    if prev is None:
        ytest[idx] = 0.0
    else:
        if ts - prev > 2 * 60 * 60:
            ytest[idx] = 1.0
        else:
            ytest[idx] = 0.0
    prev = ts 

print(np.sum(ytest))

1571.0
1727.0


In [6]:
# from CELL 24
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

rf = RandomForestClassifier(n_estimators = 100, random_state=42)

print("Training on {:,} instances for {:,} features".format(*xtrain.shape))
rf.fit(xtrain, ytrain)
ytest_pred = rf.predict(xtest)

prec, rec, f1, _ = precision_recall_fscore_support(ytest, ytest_pred)

print("[Base] Prec ", prec, "Rec", rec, "F1", f1)

Training on 4,329 instances for 9 features
[Base] Prec  [0.74353448 0.47483221] Rec [0.57944239 0.65547192] F1 [0.65131206 0.55071759]


## Radial distance

We will use a list of administrative units obtained from [GeoNames.org](http://download.geonames.org/export/dump/) for both Nigeria and Cameroon and add features as the distance to each of these points.

In [7]:
# from CELL 25
import geopy.distance

adms = list() # of coordinate tuples

with open(os.path.join(SRC_DIR, "data", "NG.txt")) as cm:
    for line in cm:
        fields = line.split("\t")
        if fields[7] == 'ADM2':
            adms.append( (fields[1], (float(fields[4]), float(fields[5]))) )
with open(os.path.join(SRC_DIR, "data", "CM.txt")) as cm:
    for line in cm:
        fields = line.split("\t")
        if fields[7] == 'ADM2':
            adms.append( (fields[1], (float(fields[4]), float(fields[5]))) )

# center for points
lat_center, lon_center = xtrain[:,-2].mean(), xtrain[:,-1].mean()
            
print("Total cities:", len(adms))

close_cities = list()
for adm_coord in adms:
    dist = geopy.distance.distance( (lat_center, lon_center), adm_coord[1])
    if dist.km < 50:
        close_cities.append(adm_coord[1])
        print("{:20}: {}".format(adm_coord[0], dist))
print("Close cities:", len(close_cities))

Total cities: 850
Bogoro              : 31.46576917722305 km
Dass                : 41.309121247543146 km
Tafawa-Balewa       : 32.22801383892469 km
Kanke               : 45.47075547698236 km
Mangu               : 35.55106011327541 km
Barkin Ladi         : 45.68022488876515 km
Jos East            : 33.46232160737422 km
Close cities: 7


Let us visualize the selected cities.

In [8]:
import folium

bird_map = folium.Map(location=(lat_center, lon_center), zoom_start=6, tiles='cartodbpositron', width=640, height=480)
[folium.CircleMarker([xtrain[idx, -2], xtrain[idx, -1]], radius=1, 
                     color='#0080bb' if ytrain[idx] else '#80bb80', fill_color='#000000').add_to(bird_map) 
    for idx in range(xtrain.shape[0])]
for idx in range(len(close_cities)):
    folium.CircleMarker(close_cities[idx], radius=2,
                       color='#bb8000', fill_color='#bb8000').add_to(bird_map)

bird_map


In [9]:
# from CELL 25
import geopy.distance

xtrain_radial = np.zeros((xtrain.shape[0], xtrain.shape[1] + len(close_cities)), dtype='float32')
xtrain_radial[:,:xtrain.shape[1]] = xtrain

for row_idx in range(xtrain.shape[0]):
    for idx, adm_coord in enumerate(close_cities):
        xtrain_radial[row_idx][xtrain.shape[1] + idx] = \
          geopy.distance.distance((xtrain[row_idx][-2], xtrain[row_idx][-1]), adm_coord).km

xtest_radial = np.zeros((xtest.shape[0], xtest.shape[1] + len(close_cities)), dtype='float32')
xtest_radial[:,:xtest.shape[1]] = xtest

for row_idx in range(xtest.shape[0]):
    for idx, adm_coord in enumerate(close_cities):
        xtest_radial[row_idx][xtest.shape[1] + idx] = \
          geopy.distance.distance((xtest[row_idx][-2], xtest[row_idx][-1]), adm_coord).km

print("Training on {:,} instances for {:,} features".format(*xtrain_radial.shape))
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(xtrain_radial, ytrain)
ytest_pred_radial = rf.predict(xtest_radial)

prec, rec, f1, _ = precision_recall_fscore_support(ytest, ytest_pred_radial)

print("[Radial] Prec ", prec, "Rec", rec, "F1", f1)

Training on 4,329 instances for 16 features
[Radial] Prec  [0.72330097 0.48667325] Rec [0.65065502 0.57093225] F1 [0.68505747 0.52544631]


This improves a little but from the visualization, we know better cities ought to be possible. (Also note, different splits of training and test set won't necessarily improve using these landmarks.)

Things to try for the workshop:

* Pick other cities from the map.
* Cluster the points into a number of groupings (three?) and pick cities close to the centre of the clusters.
* (Advanced) Use the altitude information (see its [readme](../data/GeoNames.readme.txt)) in NG.txt and CM.txt and a kriging model to incorporate altitude as a feature.
